In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
# ============================
# MODEL SEQUENTIAL SEDERHANA
# ============================

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=(28,28)),  # Mengubah gambar 28x28 menjadi vektor 1D
    keras.layers.Dense(128, activation='relu'), # Hidden layer dengan 128 neuron dan ReLU
    keras.layers.Dense(10)                      # Output layer untuk 10 kelas digit (logits)
])

print(model.summary())                          # Menampilkan ringkasan arsitektur model


/usr/local/lib/python3.12/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       100,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 101,770 (397.54 KB)

 Trainable params: 101,770 (397.54 KB)

 Non-trainable params: 0 (0.00 B)

None


In [ ]:
# ============================
# MEMBANGUN MODEL FUNCTIONAL (MULTI-OUTPUT)
# ============================

inputs = keras.Input(shape=(28,28))             # Mendefinisikan input tensor (Functional API)

flatten = keras.layers.Flatten()                # Layer flatten
dense1 = keras.layers.Dense(128, activation='relu')  # Hidden layer
dense2 = keras.layers.Dense(10)                 # Output klasifikasi digit
dense2_2 = keras.layers.Dense(1)                # Output klasifikasi kiri/kanan (binary)

x = flatten(inputs)                             # Alur data: input → flatten
x = dense1(x)                                   # → dense hidden layer
outputs = dense2(x)                             # Output pertama (digit 0–9)
outputs2 = dense2_2(x)                          # Output kedua (binary)

model2 = keras.Model(
    inputs=inputs,
    outputs=[outputs, outputs2],
    name='functional_model'
)

print(model2.summary())                         # Menampilkan ringkasan model multi-output



Model: "functional_model"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_6       │ (None, 28, 28)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_5 (Flatten) │ (None, 784)       │          0 │ input_layer_6[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_10 (Dense)    │ (None, 128)       │    100,480 │ flatten_5[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_11 (Dense)    │ (None, 10)        │      1,290 │ dense_10[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_12 (Dense)    │ (None, 1)         │        129 │ dense_10[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 101,899 (398.04 KB)

 Trainable params: 101,899 (398.04 KB)

 Non-trainable params: 0 (0.00 B)

None


In [ ]:
# ============================
# MENYALIN LAYER DARI MODEL SEQUENTIAL
# ============================

new_model = keras.models.Sequential()            # Membuat Sequential kosong
for layer in model.layers:                      # Menyalin setiap layer
  new_model.add(layer)                          # Menambahkan ke model baru


In [ ]:
# ============================
# AKSES INPUT & OUTPUT LAYER
# ============================

inputs = keras.Input(shape=(28,28))
x = new_model(inputs)

# Mengalirkan data melalui layer tertentu
for layer in new_model.layers[1:-1]:
  x = layer(x)

outputs = x                                     # Output akhir sementara


In [ ]:
# ============================
# AKSES INPUT & OUTPUT DARI MODEL FUNCTIONAL
# ============================

inputs = model2.inputs                          # Mengambil input tensor model2
outputs = model2.outputs                        # Mengambil output tensor model2

input0 = model2.layers[0].input                 # Input dari layer pertama
output0 = model2.layers[0].output               # Output dari layer pertama

print(inputs)
print(outputs)
print(input0)
print(output0)


[<KerasTensor shape=(None, 28, 28), dtype=float32, sparse=False, ragged=False, name=keras_tensor_21>]
[<KerasTensor shape=(None, 10), dtype=float32, sparse=False, ragged=False, name=keras_tensor_24>, <KerasTensor shape=(None, 1), dtype=float32, sparse=False, ragged=False, name=keras_tensor_25>]
[]
<KerasTensor shape=(None, 28, 28), dtype=float32, sparse=False, ragged=False, name=keras_tensor_21>


In [ ]:
# ============================
# TRANSFER LEARNING (VGG16)
# ============================

base_model = keras.applications.VGG16()          # Memuat model VGG16 pretrained

x = base_model.layers[-2].output                # Mengambil feature sebelum output terakhir
new_outputs = keras.layers.Dense(1)(x)           # Menambahkan output baru

new_model = keras.Model(
    inputs=base_model.inputs,
    outputs=new_outputs
)


In [ ]:
# ============================
# MODEL MULTI-OUTPUT MNIST
# ============================

inputs = keras.Input(shape=(28,28))              # Input gambar MNIST
flatten = keras.layers.Flatten()                 # Flatten layer
dense1 = keras.layers.Dense(128, activation='relu')  # Hidden layer

# Output klasifikasi digit (10 kelas)
dense2 = keras.layers.Dense(
    10, activation='softmax', name='category_output'
)

# Output klasifikasi kiri/kanan (binary)
dense3 = keras.layers.Dense(
    1, activation='sigmoid', name='leftright_output'
)


In [ ]:
x = flatten(inputs)                              # Alur data
x = dense1(x)

outputs1 = dense2(x)                             # Output digit
outputs2 = dense3(x)                             # Output binary

model = keras.Model(
    inputs=inputs,
    outputs=[outputs1, outputs2],
    name='mnist_model'
)

In [ ]:
print(model.summary())

Model: "mnist_model"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_9       │ (None, 28, 28)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_6 (Flatten) │ (None, 784)       │          0 │ input_layer_9[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_14 (Dense)    │ (None, 128)       │    100,480 │ flatten_6[1][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ category_output     │ (None, 10)        │      1,290 │ dense_14[1][0]    │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leftright_output    │ (None, 1)         │        129 │ dense_14[1][0]    │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 101,899 (398.04 KB)

 Trainable params: 101,899 (398.04 KB)

 Non-trainable params: 0 (0.00 B)

None


In [ ]:
loss1 = keras.losses.SparseCategoricalCrossentropy(from_logits=False)
loss2 = keras.losses.BinaryCrossentropy(from_logits=False)

optim = keras.optimizers.Adam(learning_rate=0.001)

metrics = {
    'category_output': ['accuracy'],             # Metric untuk klasifikasi digit
    'leftright_output': ['accuracy']             # Metric untuk klasifikasi binary
}

losses = {
    'category_output': loss1,
    'leftright_output': loss2
}

model.compile(
    loss=losses,
    optimizer=optim,
    metrics=metrics
)

In [ ]:
mnist = keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train, x_test = x_train/255.0, x_test/255.0   # Normalisasi pixel


# ============================
# MEMBUAT LABEL KEDUA (LEFT / RIGHT)
# ============================

y_leftright = np.zeros(y_train.shape, dtype=np.uint8)

for idx, y in enumerate(y_train):
  if y > 5:                                      # Digit > 5 → kelas 1
    y_leftright[idx] = 1

print(y_train.dtype, y_train[0:20])
print(y_leftright.dtype, y_leftright[0:20])


# ============================
# LABEL DALAM BENTUK DICTIONARY
# ============================

y = {
    "category_output": y_train,                  # Label digit 0–9
    "leftright_output": y_leftright              # Label binary
}


uint8 [5 0 4 1 9 2 1 3 1 4 3 5 3 6 1 7 2 8 6 9]
uint8 [0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 1 1 1]


In [ ]:
model.fit(x_train, y=y, epochs=5, batch_size=64, verbose=2)

Epoch 1/5
938/938 - 8s - 8ms/step - category_output_accuracy: 0.9144 - category_output_loss: 0.3052 - leftright_output_accuracy: 0.9330 - leftright_output_loss: 0.1840 - loss: 0.4893
Epoch 2/5
938/938 - 4s - 4ms/step - category_output_accuracy: 0.9597 - category_output_loss: 0.1382 - leftright_output_accuracy: 0.9682 - leftright_output_loss: 0.0941 - loss: 0.2323
Epoch 3/5
938/938 - 3s - 4ms/step - category_output_accuracy: 0.9709 - category_output_loss: 0.0985 - leftright_output_accuracy: 0.9753 - leftright_output_loss: 0.0736 - loss: 0.1722
Epoch 4/5
938/938 - 5s - 5ms/step - category_output_accuracy: 0.9769 - category_output_loss: 0.0763 - leftright_output_accuracy: 0.9797 - leftright_output_loss: 0.0616 - loss: 0.1378
Epoch 5/5
938/938 - 4s - 4ms/step - category_output_accuracy: 0.9814 - category_output_loss: 0.0603 - leftright_output_accuracy: 0.9823 - leftright_output_loss: 0.0517 - loss: 0.1120


In [ ]:
predictions = model.predict(x_test)              # Menghasilkan dua output
len(predictions)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


2

In [ ]:
prediction_category = predictions[0]             # Output digit
prediction_lr = predictions[1]                   # Output left/right

pr_cat = prediction_category[0:20]
prediction_lr = prediction_lr[0:20]

labels_cat = np.argmax(pr_cat, axis=1)            # Konversi softmax → label digit
labels_lr = np.array([1 if p >= 0.5 else 0 for p in prediction_lr])

In [ ]:
print(y_test[0:20])
print(labels_cat)
print(labels_lr)

[7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4]
[7 2 1 0 4 1 4 9 6 9 0 6 9 0 1 5 9 7 3 4]
[1 0 0 0 0 0 0 1 1 1 0 1 1 0 0 0 1 1 0 0]


In [ ]:
# ============================================================
# RINGKASAN KODE
# ============================================================
#
# Kode ini mendemonstrasikan:
# 1. Perbedaan Sequential API dan Functional API di Keras
# 2. Cara membangun model multi-output (multi-task learning)
# 3. Satu input (MNIST image) menghasilkan dua output:
#    - Klasifikasi digit 0–9
#    - Klasifikasi binary (digit <=5 atau >5)
# 4. Penggunaan loss & metrics berbeda untuk tiap output
# 5. Penggunaan dictionary pada model.compile() dan model.fit()
# 6. Post-processing output softmax dan sigmoid
#
# Model ini adalah contoh klasik MULTI-TASK LEARNING
# dengan shared feature extraction.
